In [ ]:
%pip install plotly

In [ ]:
# Environment Setup (may need to run above)
# Data
import pandas as pd
# Plotting
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
py.init_notebook_mode()

In [ ]:
# Import data
# Needs updating once predictions are generated
df = pd.read_csv('../data/ParlVote_concat.csv') # replace with predictions dataset
elections_df = pd.read_csv('datasets/UK_Elections.csv')

# Fix date in the predicted data 
df['Date'] = df['debate_id'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df['Date'] = pd.to_datetime(df['Date'])
df['Sentiment'] = df['vote'].map({0: 'Negative', 1: 'Positive'})

# Preprocess the main dataset
df['Date'] = pd.to_datetime(df['Date'])
df['YearMonth'] = df['Date'].dt.to_period('M')

# Group by YearMonth and Sentiment, count occurrences
sentiment_counts = df.groupby(['YearMonth', 'Sentiment']).size().unstack(fill_value=0)

# Add Percentages
sentiment_counts['Total'] = sentiment_counts['Negative'] + sentiment_counts['Positive']
sentiment_counts['Negative %'] = (sentiment_counts['Negative'] / sentiment_counts['Total']) * 100
sentiment_counts['Positive %'] = (sentiment_counts['Positive'] / sentiment_counts['Total']) * 100

# Preprocess the elections dataset
elections_df['Date'] = pd.to_datetime(elections_df['Date'], format='%d/%m/%Y')
elections_df = elections_df[elections_df['Date'] >= '1997-05-01']
elections_df = elections_df[elections_df['Date'] <= '2019-10-30']

In [ ]:
# Create figures
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.8, 0.2],
    specs=[[{"type": "scatter"}, {"type": "pie"}]],
    subplot_titles=(
        "Sentiment Over Time",
        "Sentiment Breakdown (All Time)"
        )
)

# Line Chart
# Sentiment
for sentiment in ['Positive %', 'Negative %']:
    fig.add_trace(
        go.Scatter(x=sentiment_counts.index.to_timestamp(), y=sentiment_counts[sentiment], 
                   name=sentiment.capitalize(), mode='lines'),
        row=1, col=1
    )
# Election Dates
for _, election in elections_df.iterrows():
    fig.add_vline(x=election['Date'], line_dash="dash", line_color="gray", row=1, col=1)
    fig.add_annotation(
        x=election['Date'], y=10, text=election['Party_forming_government'],
        showarrow=False, textangle=-90, yshift=0, xshift = -10,
        xref='x', yref='y',  # Explicitly tie to x and y axes
        row=1, col=1
    )

# Pie Chart
total_pos = sentiment_counts['Positive'].sum()
total_neg = sentiment_counts['Negative'].sum()
fig.add_trace(
    go.Pie(labels=['Positive', 'Negative'], values=[total_pos, total_neg], 
           name="Sentiment Breakdown (All Time)", hole=0.5),
    row=1, col=2
)

# Update Layout
fig.update_layout(
    title_text= "Zilog Z80",
    template="plotly_dark",
    margin=dict(r=20, t=50, b=50, l=20),
    showlegend=False,
    xaxis_rangeslider_visible=True
)

fig.show()